<a href="https://colab.research.google.com/github/guarnieri0/GAIN/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import librerie**

In [251]:
import pandas as pd 
import json
import datetime
from sklearn.preprocessing import LabelEncoder

**Reading Dataset**

In [252]:
df = pd.read_json("today_dataset.json")
df_past = pd.read_json("past_dataset.json")

**Deleting useless columns**

In [253]:
df = df.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)
df_past = df_past.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)

**Retrieving the Error Code and Error description**

In [254]:
df_splitted= df['compareInfo'].str.split(pat=',', expand=True)
df_splitted = df_splitted.drop([0,1], axis = 1)
error_code = df_splitted.iloc[:,0].to_list()
error_description = df_splitted.iloc[:,1].to_list()
df.insert(5, 'error_description', error_description)
df.insert(6,'error_code',error_code)

df_splitted_past= df_past['compareInfo'].str.split(pat=',', expand=True)
df_splitted_past = df_splitted_past.drop([0,1], axis = 1)
error_code = df_splitted_past.iloc[:,0].to_list()
error_description = df_splitted_past.iloc[:,1].to_list()
df_past.insert(5, 'error_description', error_description)
df_past.insert(6,'error_code',error_code)

**Deleting redundant columns**

In [255]:
df = df.drop(['compareInfo','reportTitle'], axis = 1)
df_past = df_past.drop(['compareInfo','reportTitle'], axis = 1)

**Convert timestamp in year/month/day**

---



In [256]:
df['timestamp'] = df['timestamp'].dt.date
df_past['timestamp'] = df_past['timestamp'].dt.date


In [257]:
print(df.columns)
print(df_past.columns)

Index(['testName', 'reportId', 'timestamp', 'error_description', 'error_code'], dtype='object')
Index(['testName', 'reportId', 'timestamp', 'error_description', 'error_code'], dtype='object')


In [258]:
df['error_description'] = df['error_description'].fillna('None')
df['error_code'] = df['error_code'].fillna('None')
#df['error_description'] = df['error_description'].fillna('None')


In [259]:
encoder_df_error_code = LabelEncoder()
encoder_df_past_error_code = LabelEncoder()
encoder_df_error_description = LabelEncoder()
encoder_df_past_error_description = LabelEncoder()

df['error_code'] = encoder_df_error_code.fit_transform(df['error_code'])
df['error_description'] = encoder_df_error_description.fit_transform(df['error_description'])
df_past['error_code'] = encoder_df_past_error_code.fit_transform(df_past['error_code'])
df_past['error_description'] = encoder_df_past_error_description.fit_transform(df_past['error_description'])

In [260]:
print(df_past.max())

testName             test_verification
reportId               MNDRACOVAL-9255
timestamp                   2022-10-06
error_description                   32
error_code                          19
dtype: object
